# BERRIEN Samuel, LANCO Antoine, DE OLIVEIRA Joe

# EXO1


Partons de l'exercice 3 (à faire de préférence avant)

1) Expliciter et expliquer l'hypothèse faite lors de la construction du classifieur. En particulier, chaque pixel est considéré comme une     variable aléatoire et donc une image comme la réalisation d'un vecteur aléatoire. Quelle hypothèse est faite sur la dépendance entre les pixels ? Quel est l'impact sur la matrice de covariance ?

- On fera l’hypothèse que les données (VA) sont indépendantes et identiquement distribuées. Chaque image représente une expérience aléatoire
- On fait donc l’hypothèse que les pixels sont fortement indépendants entre eux . Cela implique que la matrice de covariance n’aura de valeur forte qu’en diagonale : les pixels ne seront corrélés qu’avec eux même.
- Le classifieur dit naïf fait donc l’hypothèse que tout indépendant


2) Proposer une mesure permettant d'évaluer cette hypothèse.

![alt text](./index.png)
La matrice de covariance peut se résumer à une diagonale : chaque pixel n'est corrélé qu'avec lui-même

3) Évaluons cette hypothèse en considérant comme voisinage pour chaque pixel, son voisin de gauche et de droite.

- Une valeur de la réalisation du vecteur de V.As n’est plus un pixel simple mais l’ensemble contenant lui-même et celui de gauche et droite



4) Si nous faisions l'hypothèse qu'un pixel ne dépend que de ses voisins immédiats de gauche et de droite, écrire les paramètres à estimer et comment les estimer. On pourra se contenter de dépendances simples: si une image est un vecteur, un pixel (une composante du vecteur) dépend du pixel avant et après dans le vecteur; pour les pixels sur les bords du vecteurs peuvent être ignoré si besoin.

$ P(Y=y | X=x)\,\alpha\,P(X = x | Y=y) * P(Y=y) $

$ P(X = x | Y=y) * P(Y=y) = (\prod_{i = 1}^{d} \pi{_{i, y, x_{i-1}, x_{i+1}}}^{x_{i}}(1-\pi_{i, y, x_{i-1}, x_{i+1}})^{1-x_{i}})P(Y = y) $

Avec :
$ \pi_{i,y,a,b} = \frac{\left | \left \{ elements\,de\,la\,classe\,y\,dont\,x_{i}\,est\,vrai\,et\,x_{i-1}=a\,et\,x_{i+1}=b \right \} \right |}{\left | \left \{ elements\,de\,la\,classe\,y\,tels\,que\,x_{i-1}=a\,et\,x_{i+1}=b \right \} \right |} $

5) BONUS: Coder et expérimenter ce modèle
    
Pour répondre à l'avant dernière question, une solution est de considérer que la probabilité d'une image est la probabilité jointe sur l'ensemble des pixels. Sans hypothèse, cette probabilité se décompose avec la règle de la chaine. Puis il est possible d'introduire les hypothèses d'indépendance.

In [1]:
import pickle, gzip
import numpy as np
import math
import operator
# Load the dataset
f = gzip.open('./mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
p = u.load()
train_set, valid_set, test_set = p
labels = train_set[1]
# K = nombre de classes
K = 10  
    
# Calcule priors
priors = {}
for k in range(K):
    priors[k] = np.log(len(labels[labels == k]) / len(labels))
    
# binarise les images
def mk_bin(images):
    res = []
    for img in images:
        tmp = np.zeros(img.shape)
        tmp[img > 0.5] = 1
        res.append(tmp)
    return np.asarray(res)

images = mk_bin(train_set[0])
labels = train_set[1]

images_test = mk_bin(test_set[0])
labels_test = test_set[1]

# probabilité d'un pixel selon ses voisins de gauche et droite
beta = np.ndarray((K, 784, 2, 2))

# calcul de l'estimation selon les pixels de gauche et droite
for y, img in enumerate(images):
    for i in range(1, len(img) - 1):
        x_i_m = int(img[i - 1]) # pixel de gauche
        x_i_p = int(img[i + 1]) # pixel de droite
        beta[labels[y],i,x_i_m,x_i_p] += img[i] # On ajoute le pixel en fonction de ses voisins
        
# On transforme en probabilité
for k in range(K):
    beta[k,:,0,0] /= beta[k,:,0,0].sum()
    beta[k,:,1,0] /= beta[k,:,1,0].sum()
    beta[k,:,0,1] /= beta[k,:,0,1].sum()
    beta[k,:,1,1] /= beta[k,:,1,1].sum()


# Prediction avec la formule naive bayes
def predict_naive_bayes(img, beta):
    naive_bayes = {}
    for k in range(K):
        # P(Y = y)
        p_Y_eq_y = priors[k]
        tmp = 0
        for i in range(1, len(img) - 1):
            # On récupère les pixels gauche et droit
            x_i_m = int(img[i - 1])
            x_i_p = int(img[i + 1])
            # On récupère la probabilité associée
            piiy = beta[k,i,x_i_m,x_i_p]
            # On ne veut pas de probabilité nulle pour le log
            if piiy == 0:
                continue
            # Calcul de la probabilité
            tmp += img[i] * math.log(piiy) + (1 - img[i]) * math.log(1-piiy)
        naive_bayes[k] = tmp + p_Y_eq_y
    return naive_bayes


# Evaluation du modele
loss = 0.
for i in range(len(test_set[0][:1000])):
    pred = predict_naive_bayes(test_set[0][i], beta)
    res = max(pred.items(), key=operator.itemgetter(1))[0]
    loss += 1 if res != test_set[1][i] else 0
loss /= len(test_set[1])
print("% de mauvaises réponses :",loss * 100)



# lissage
alpha = 1e-9
beta = np.ndarray((10,784,2,2))
for y, img in enumerate(images):
    for i in range(1, len(img) - 1):
        x_i_m = int(img[i - 1])
        x_i_p = int(img[i + 1])
        beta[labels[y],i,x_i_m,x_i_p] += img[i] + alpha
for k in range(10):
    beta[k,:,0,0] /= beta[k,:,0,0].sum() + K * alpha
    beta[k,:,1,0] /= beta[k,:,1,0].sum() + K * alpha
    beta[k,:,0,1] /= beta[k,:,0,1].sum() + K * alpha
    beta[k,:,1,1] /= beta[k,:,1,1].sum() + K * alpha
    
# Evaluation du modele (lissage)
loss = 0.
for i in range(len(test_set[0][:1000])):
    pred = predict_naive_bayes(test_set[0][i], beta)
    res = max(pred.items(), key=operator.itemgetter(1))[0]
    loss += 1 if res != test_set[1][i] else 0
loss /= len(test_set[1])
print("% de mauvaises réponses (lissage avec α =", alpha, ") :",loss * 100)

% de mauvaises réponses : 7.32
% de mauvaises réponses (lissage avec α = 1e-09 ) : 4.71


# EXO2

Repartons du TP sur Bayésien Naif Gaussien appliqué aux images MNIST.

Le classifieur vu en TP considère la réalisation un pixel conditionnellement à une classe comme une variable aléatoire gaussienne.

- Prenez les images de la classe 4, prendre 5 pixels avec une forte variance et tracer pour chacun la répartition des valeurs avec un histogramme.


- Faites la même chose pour la classe 5.


- Commenter les résultats obtenus

Supposons maintenant que la valeur d'un pixel conditionnellement à sa classe est en fait une variable aléatoire dont la distribution est donné par le mélange de 2 gaussiennes:

        Xi|Y=y∼αi,y,1N(μi,y,1,σi,y,1)+αi,y,2N(μi,y,2,σi,y,2)
        p(xi|y)=αi,y,1p1(x|y)+αi,y,2p2(x|y),

avec p1(x|y)=N(μi,y,1,σi,y,1). La distribution est en fait l'interpolation linéaire de 2 gaussiennes, et :
    αi,y,1+αi,y,2=1
La question qui se pose est comment calculer les paramètres de ce nouveau modèle pour une classe y donnée, à savoir :

        (αi,y,1,αi,y,2,μi,y,1,μi,y,2,σi,y,1,σi,y,2)

et nous allons voir comment au travers des questions suivantes.

Prenons par exemple les images de la classe 5 et un pixel du centre de l'image (par exemple un des pixels que vous avez sélectionné dans une des questions précédentes). Dans le cas à une seule gaussienne, chaque réalisation contribue à l'estimation des statistiques nécessaires (μ,σ).
Avec 2 gaussiennes, on ne sais pas à gaussienne, une observation va contribuer. Mais on peut calculer cette contribution, si on connait et fixe les paramètres.

- Proposer une fonction permettant de calculer la contribution d'une réalisation à chacune des gaussienne. Sachant que la somme des deux contributions est 1 et qu'elles dépendent de αi,y,1,αi,y,1,p1,p2.


- Une fois connue cette contribution, il est possible de mettre à jour les paramètres, puisque l'on sait pour chaque réalisation, de combien elle participe à chaque gaussienne. Une fois les contribution de chaque réalisation calculée sur les données d'apprentissage, écrire la formule d'estimation des paramètres pour

        (αi,y,1,αi,y,2,μi,y,1,μi,y,2,σi,y,1,σi,y,2).

Ainsi, si les paramètres sont connus, il est possible de calculer les contributions de chaque exemple, puis de recalculer les paramètres. Ainsi l'algorithme d'apprentissage est itératif:

- 0 : initialisation des contributions (tirage au hasard entre les deux gaussiennes)
- 1 : Calcul des paramètres.
- 2 : Calcul des contributions et retour à l'étape 1

Cet algorithme s'appelle E.M, et vous allez l'implémenter avec 2 fonctions à coder et à tester.

- Partir des données MNIST et les images de la classe 5, estimer les paramètres du mélange, visualiser la distribution obtenues pour les pixels sélectionnés et comparer cette fonction à l'histogramme des valeurs.


- Recommencer avec plusieurs initialisations aléatoires pour comparer les résultats. Sont-ils stables ?


- Faire de même avec la classe 4


- BONUS : coder un classifier qui utilise un mélange de 2 gaussiennes pour modéliser Xi|Y , évaluer ses performances et comparer avec le classifieur de base.

# Réponses

- Proposer une fonction permettant de calculer la contribution d'une réalisation à chacune des gaussienne. Sachant que la somme des deux contributions est 1 et qu'elles dépendent de αi,y,1,αi,y,1,p1,p2.

$ \LARGE p_{1}(X_{i}\,|\,Y) = \frac{\alpha_{1,i,y}\,\mathcal{N}(\mu_{1,i,y}, \sigma_{1,i,y})}{\alpha_{1,i,y}\,\mathcal{N}(\mu_{1,i,y}, \sigma_{1,i,y}) + \alpha_{2,i,y}\,\mathcal{N}(\mu_{2,i,y}, \sigma_{2,i,y})} $

$ \LARGE p_{2}(X_{i}\,|\,Y) = 1 - p_{1}(X_{i}\,|\,Y) $

- Une fois connue cette contribution, il est possible de mettre à jour les paramètres, puisque l'on sait pour chaque réalisation, de combien elle participe à chaque gaussienne. Une fois les contribution de chaque réalisation calculée sur les données d'apprentissage, écrire la formule d'estimation des paramètres pour
    (αi,y,1,αi,y,2,μi,y,1,μi,y,2,σi,y,1,σi,y,2).
    
$ \LARGE \mu_{1,i,y} = \frac{\sum_{j=1}^{N}p_{1}(X_{i}^{(j)}\,|\,Y)X_{i}^{(j)}}{\sum_{j=1}^{N}p_{1}(X_{i}^{(j)}\,|\,Y)} $

$ \LARGE \sigma_{1,i,y} = \frac{\sum_{j=1}^{N}p_{1}(X_{i}^{(j)}\,|\,Y)(X_{i}^{(j)}-\mu_{1,i,y})^{2}}{\sum_{j=1}^{N}p_{1}(X_{i}^{(j)}\,|\,Y)} $

où $ X_{i}^{(j)} $ est les i-eme pixel de la j-eme image

In [3]:
import pickle, gzip
import numpy as np
import math
import operator
import random
np.seterr(divide='ignore', invalid='ignore')
# Load the dataset
f = gzip.open('./mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
p = u.load()
train_set, valid_set, test_set = p

# K = nombre de classes

# Récupération des images et des labels
images = train_set[0][:100]
labels = train_set[1][:100]


means = {}
variances = {}
priors = {}

means1 = np.random.rand(10,784)
means2 = np.random.rand(10,784)
var1   = np.random.rand(10,784)
var2   = np.random.rand(10,784)
alpha  = np.empty((10,784))
alpha[:]  = 0.5




for i in range(0, 10):
    means[i]     = images[labels==i].mean(axis=0)
    variances[i] = images[labels==i].var(axis=0)
    priors[i]    = len(images[labels==i])/len(images)
    

        
def computePosteriors(image,means,variances,priors):
    res = np.zeros(10)
    for lbl in range(10):
        mean = means[lbl]
        sigma2 = variances[lbl]
        non_null = sigma2!=0
        
        scale = 0.5*np.log(2*sigma2[non_null]*math.pi)
        expterm = -0.5*np.divide(np.square(image[non_null]-mean[non_null]), sigma2[non_null])
        
        llh = (expterm - scale).sum()
        post = llh + np.log(priors[lbl])
        res[lbl]=post
    return res


def computeGauss(means,var,x):
    
    scale = 1/math.sqrt(2*math.pi*var)
    expterm = math.exp(-(((x-means)**2)/(2*var)))    
    return scale*expterm



def winrate(images_test,labels_test):
    res = 0
    for i in range(len(images_test)):
        l = list(computePosteriors(images_test[i],means,variances,priors))
        res += 1 if labels_test[i] != l.index(max(l)) else 0
        
    return 1-(res/len(images_test))

images_test = test_set[0]
labels_test = test_set[1]

def getMax(array,nb):
    res = np.zeros(nb)
    for i in range(nb):
        max = np.argmax(array)
        res[i] = max
        array[max] = 0
    return res
    
    
def getline(array,line):
    res = np.zeros(int(array.size/array[0].size))
    for i in range(res.size):
        res[i] = array[i][int(line)]
            
    return res
    
    

#print("Win Rate : " + str(winrate(images_test,labels_test)*100) + " %")

import matplotlib.pyplot as plt

'''
for j in range(3,4):
    var = variances[j]
    M = getMax(var,5)
    print("------------"+str(j)+"-------------")
    for i in M:    
        plt.hist(getline(images[labels == j], i), log=True)
        plt.show()
'''    
for it in range(5):
    #---------contribution-------------
    p1 = np.empty((10,len(images),784))
    for i in range(10):
        for j in range(len(images)):
            for k in range(784):
                p1[i][j][k] = (alpha[i][k]*computeGauss(means1[i][k],var1[i][k],images[j][k])) / \
                    ((alpha[i][k]*computeGauss(means1[i][k],var1[i][k],images[j][k]))+\
                    ((1-alpha[i][k])*computeGauss(means2[i][k],var2[i][k],images[j][k])))

    p2 = 1-p1

    #--------maj--------

    means1 =np.true_divide((p1*images).sum(axis=1),p1.sum(axis=1))
    means2 = np.true_divide((p2*images).sum(axis=1),p2.sum(axis=1))


    for i in range(10):
        for j in range(len(images)):
            for k in range(784):
                var1[i][k] += p1[i][j][k]*((images[j][k] - means1[i][k])**2)
                var2[i][k] += p2[i][j][k]*((images[j][k] - means2[i][k])**2)

    var1 = np.true_divide(var1,p1.sum(axis=1))
    var2 = np.true_divide(var2,p2.sum(axis=1))

    alpha  = np.true_divide(p1.sum(axis=1),len(images))

print("fin")

/home/samuel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:130: RuntimeWarning: overflow encountered in true_divide


fin


# EXO3 Bayésien naïf discret

Reprendre la même tâche mais en binarizant les images. Pour cela, le modèle de chaque pixel est une binomiale. 

## TODO
- Quels sont les paramètres à estimer pour chaque classe ? 
- Comment les stockés ? 
- Implémenter l'estimation, puis l'inférence. (On peut dans un premier temps réutiliser l'implémentation existante de Sklearn)


In [1]:
import pickle, gzip
import numpy as np
import math
import operator
# Load the dataset
f = gzip.open('./mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
p = u.load()
train_set, valid_set, test_set = p
labels = train_set[1]
# K = nombre de classes
K = 10  
    
# Calcule priors
priors = {}
for k in range(K):
    priors[k] = np.log(len(labels[labels == k]) / len(labels))
    
# binarise les images
def mk_bin(images):
    res = []
    for img in images:
        tmp = np.zeros(img.shape)
        tmp[img > 0.5] = 1
        res.append(tmp)
    return np.asarray(res)

images = mk_bin(train_set[0])
labels = train_set[1]

images_test = mk_bin(test_set[0])
labels_test = test_set[1]

beta = np.ndarray((K, 784))


# calcul de l'estimation
for y, img in enumerate(images):
    beta[labels[y]] += img
for k in range(K):
    beta[k] /= beta[k].sum()
    
# Prediction avec la formule naive bayes
def predict_naive_bayes(img, beta):
    naive_bayes = {}
    for k in range(K):
        p_Y_eq_y = priors[k]
        non_null = beta[k] != 0
        xi = img[non_null]
        piiy = beta[k][non_null]
        tmp = (xi * np.log(piiy) + (1-xi) * np.log(1-piiy) ).sum()
        naive_bayes[k] = tmp + p_Y_eq_y
    return naive_bayes

# Evaluation du modele
loss = 0.
for i in range(len(test_set[0])):
    pred = predict_naive_bayes(test_set[0][i], beta)
    res = max(pred.items(), key=operator.itemgetter(1))[0]
    loss += 1 if res != test_set[1][i] else 0
loss /= len(test_set[1])
print("% de mauvaises réponses :",loss * 100)

# lissage
alpha = 1e-9
beta = np.ndarray((10,784))
for y, img in enumerate(images):
    beta[labels[y]] += img + alpha
for k in range(10):
    beta[k] /= beta[k].sum() + K * alpha
    
# Evaluation du modele (lissage)
loss = 0.
for i in range(len(test_set[0])):
    pred = predict_naive_bayes(test_set[0][i], beta)
    res = max(pred.items(), key=operator.itemgetter(1))[0]
    loss += 1 if res != test_set[1][i] else 0
loss /= len(test_set[1])
print("% de mauvaises réponses (lissage avec α = 1e-9 ) :",loss * 100)

% de mauvaises réponses : 17.97
% de mauvaises réponses (lissage avec α = 1e-9 ) : 15.959999999999999
